In [1]:
from sympy import *
import numpy as np
import dimod
from dimod.reference.samplers import ExactSolver
import pandas as pd
init_printing(use_unicode=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
lattice = Matrix([[1, 2, 2], [2, 2, 2], [2, 2, 3]])
lattice.det()

In [3]:
(d1, d2) = shape(lattice)

In [20]:
def calculate_gram_matrix(lattice):
    (d1, d2) = shape(lattice)
    return Matrix([[lattice.row(i)*lattice.row(j).T for i in range(0, d1)] for j in range(0,d2)])

In [21]:
G = calculate_gram_matrix(lattice)
G

⎡9   10  12⎤
⎢          ⎥
⎢10  12  14⎥
⎢          ⎥
⎣12  14  17⎦

In [22]:
def max_num_qubits_binary(lattice):
    return int(np.ceil((3.0/2.0)*shape(lattice)[0]*np.log2(shape(lattice)[0])+shape(lattice)[0]+np.log2(float(np.absolute(lattice.det())))))

In [23]:
max_num_qubits_binary(lattice)

In [24]:
def max_num_qubits_ternary(lattice):
    return int(1+np.floor(np.log(np.ceil((3.0/2.0)*shape(lattice)[0]*np.log2(shape(lattice)[0])+shape(lattice)[0]+np.log2(float(np.absolute(lattice.det())))))/log(2.0)))

In [25]:
max_num_qubits_ternary(lattice)

In [26]:
shape_qubits_ternary = [num_qudits, dim_qudits] = [d1, max_num_qubits_ternary(lattice)*2]
print(shape_qubits_ternary)

[3, 8]


In [27]:
s = []
s = symbols('s:'+str(shape_qubits_ternary[0]*shape_qubits_ternary[1]))
s

In [12]:
num_qubits = len(s)
shape_qubits = shape_qubits_ternary

In [13]:
f = []
for i in range(0,shape_qubits[0]):
    expr = s[i*shape_qubits[1]] + s[i*shape_qubits[1]+1]
    for j in range(2, shape_qubits[1],2):
        expr += int(3**(j/2))*(s[i*shape_qubits[1]+j] + s[i*shape_qubits[1]+j+1])
    expr -= int(2*3**(np.floor(shape_qubits[1]/2.0)))
    f.append(expr)
f

In [14]:
H = 0
for i in range(0,shape_qubits[0]):
    for j in range(0,shape_qubits[0]):
        H += G[i,j]*f[i]*f[j]
H

In [15]:
replacements = [(s[i]**2, s[i]) for i in range(num_qubits)]
H = H.expand().subs(replacements)
H

In [16]:
Q = H.as_coefficients_dict()

C = 0
h = {}
J = {}
Q_ = iter(Q)
for i in range(0, len(Q)):
    key = next(Q_)
    if i == 0:
        C = Q[key]
    elif i > 0 and i < num_qubits + 1:
        h[int(str(key)[1:])] = Q[key]
    else:
        keys = str(key).split('*')
        J[(int(keys[0][1:]), int(keys[1][1:]))] = Q[key]

        
print('Constant:\n',C)
print('\nLinear Terms:\n',h)
print('\nQuadratic Terms:\n',J)

Constant:
 72900

Linear Terms:
 {10: -12960, 11: -12960, 4: -10287, 5: -10287, 8: -4464, 9: -4464, 2: -3519, 3: -3519, 6: -1504, 7: -1504, 0: -1183, 1: -1183}

Quadratic Terms:
 {(0, 1): 10, (0, 6): 12, (0, 7): 12, (1, 6): 12, (1, 7): 12, (6, 7): 16, (0, 2): 30, (0, 3): 30, (1, 2): 30, (1, 3): 30, (0, 8): 36, (0, 9): 36, (1, 8): 36, (1, 9): 36, (2, 6): 36, (2, 7): 36, (3, 6): 36, (3, 7): 36, (6, 8): 48, (6, 9): 48, (7, 8): 48, (7, 9): 48, (0, 4): 90, (0, 5): 90, (1, 4): 90, (1, 5): 90, (2, 3): 90, (0, 10): 108, (0, 11): 108, (1, 10): 108, (1, 11): 108, (2, 8): 108, (2, 9): 108, (3, 8): 108, (3, 9): 108, (4, 6): 108, (4, 7): 108, (5, 6): 108, (5, 7): 108, (10, 6): 144, (10, 7): 144, (11, 6): 144, (11, 7): 144, (8, 9): 144, (2, 4): 270, (2, 5): 270, (3, 4): 270, (3, 5): 270, (10, 2): 324, (10, 3): 324, (11, 2): 324, (11, 3): 324, (4, 8): 324, (4, 9): 324, (5, 8): 324, (5, 9): 324, (10, 8): 432, (10, 9): 432, (11, 8): 432, (11, 9): 432, (4, 5): 810, (10, 4): 972, (10, 5): 972, (11, 4): 9

In [17]:
bqm = dimod.BinaryQuadraticModel(h, J, C, 'BINARY')
sampler = ExactSolver()
response = sampler.sample(bqm)
response.to_pandas_dataframe(sample_column=False).sort_values('energy')

,0,1,2,3,4,5,6,7,8,9,10,11,energy,num_occurrences
2730,1,1,1,1,1,1,1,1,1,1,1,1,19600.0,1
1365,1,0,1,1,1,1,1,1,1,1,1,1,20221.0,1
3413,0,1,1,1,1,1,1,1,1,1,1,1,20221.0,1
2389,1,1,1,1,1,1,1,0,1,1,1,1,20392.0,1
2901,1,1,1,1,1,1,0,1,1,1,1,1,20392.0,1
682,0,0,1,1,1,1,1,1,1,1,1,1,20852.0,1
1706,1,0,1,1,1,1,1,0,1,1,1,1,21025.0,1
1194,1,0,1,1,1,1,0,1,1,1,1,1,21025.0,1
3754,0,1,1,1,1,1,1,0,1,1,1,1,21025.0,1
3242,0,1,1,1,1,1,0,1,1,1,1,1,21025.0,1
